In [2]:
import numpy as np
import pandas as pd
import os
from datetime import date
import sqlalchemy
import pymysql
import re
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)

import warnings    # to avoid warning during executions
warnings.filterwarnings("ignore")

In [3]:
file_name = '..\..\..\dataExport\LOGdata\log_2022_4_20.csv'
log_rawDF = pd.read_csv(file_name, sep='*')

In [4]:
log_rawDF.head(5)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
0,2021-4-20 13:0:13,1618916413,613129,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN
1,2021-4-20 13:0:57,1618916457,9327,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN
2,2021-4-20 13:1:38,1618916498,812633,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN
3,2021-4-20 13:2:20,1618916540,214501,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN
4,2021-4-20 13:3:3,1618916583,212528,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN


In [5]:
feed_temp_ins4 = log_rawDF.query('Time >= 1618561800 and Time <= 1625228995 and Unit =="FPC14" and Parameter == "program"')

In [6]:
feed_temp_ins4.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
6,2021-4-20 13:3:58,1618916638,271269,3,FPC14,program,step ID - Changed (new and old value),4_WaterAcid,5_empty
7,2021-4-20 13:4:1,1618916641,101959,3,FPC14,program,start at - Changed (new and old value),sellected step at...,first step
8,2021-4-20 13:4:6,1618916646,386245,3,FPC14,program,procedure running - Changed,not active,active
9,2021-4-20 13:4:7,1618916647,639887,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
10,2021-4-20 13:4:7,1618916647,642814,2,FPC14,program,Step 4_WaterAcid started,NaN,NaN


In [7]:
programID_DF = feed_temp_ins4[feed_temp_ins4['Message'].str.contains('PrgID', regex=False, case=False, na=False)]

In [8]:
programID_DF.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
1436,2021-4-20 22:0:5,1618948805,125813,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
25922,2021-4-21 22:0:5,1619035205,342017,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
27652,2021-4-22 10:0:5,1619078405,339214,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
29425,2021-4-22 22:0:5,1619121605,341318,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
30376,2021-4-23 10:0:5,1619164805,351891,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN


In [9]:
programID_DF['Message'].unique()

array(['program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65',
       'program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 50'],
      dtype=object)

In [10]:
StepID_DF = feed_temp_ins4[feed_temp_ins4['Message'].str.contains('Step', regex=False, case=False, na=False)]

In [11]:
StepID_DF.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
6,2021-4-20 13:3:58,1618916638,271269,3,FPC14,program,step ID - Changed (new and old value),4_WaterAcid,5_empty
10,2021-4-20 13:4:7,1618916647,642814,2,FPC14,program,Step 4_WaterAcid started,NaN,NaN
16,2021-4-20 13:4:25,1618916665,180599,2,FPC14,program,Step 4_WaterAcid started,NaN,NaN
146,2021-4-20 14:4:1,1618920241,169613,2,FPC14,program,Step 5_empty started,NaN,NaN
1438,2021-4-20 22:0:6,1618948806,648291,2,FPC14,program,Step 1_Harvest started,NaN,NaN


In [12]:
StepID_DF['Message'].unique()

array(['step ID - Changed (new and old value)',
       'Step 4_WaterAcid started', 'Step 5_empty started',
       'Step 1_Harvest started', 'Step 2_SiliciumHCl started',
       'Step 3_CompleteNutrients started',
       'Record ProgramID = F_ThalaPseu_HalfNutr  StepID = 1_Harvest saved (old and new record)EmptyReactorHarvestVolumeProductCIPFeedVolumeStopCirculationDuringFeedPneumaticDuringFeedPneumaticTimeDripTimeAerationAfterDripTimeDripCyclesOzoneDuringWaitwaitingTimeHarvestTriggerLastFeedStartedTriggerFeedTriggerStepTrigger',
       'Step 1_water started', 'Step 2_Oxonia 2% started',
       'Step step1 started', 'Step step2 started', 'Step step3 started'],
      dtype=object)

In [13]:
program_DF = feed_temp_ins4[feed_temp_ins4['Message'].str.contains('Program', regex=False, case=False, na=False)]

In [14]:
program_DF.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
9,2021-4-20 13:4:7,1618916647,639887,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
12,2021-4-20 13:4:11,1618916651,125940,2,FPC14,program,_======program F_ThalaPseu_HalfNutr stopped before ended!!===========,NaN,NaN
15,2021-4-20 13:4:25,1618916665,177004,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
172,2021-4-20 14:5:54,1618920354,625413,2,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
1436,2021-4-20 22:0:5,1618948805,125813,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN


In [15]:
L = ['program','Program']
just_programDF = feed_temp_ins4[feed_temp_ins4['Message'].str.startswith(tuple(L))]

In [16]:
just_programDF['Message'].unique()

array(['Program F_ThalaPseu_HalfNutr started',
       'Program F_ThalaPseu_HalfNutr ended',
       'program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65',
       'program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 50',
       'Program CleanOxonia_0.5% started',
       'Program waterSpoeling started', 'Program waterSpoeling ended',
       'Program Clean feed line base started',
       'Program Clean feed line base ended'], dtype=object)

In [17]:
#df[~df.C.str.contains("XYZ")]

only_feed = just_programDF[just_programDF["Message"].str.contains("CleanOxonia|waterSpoeling|line")==False]

In [18]:
only_feed['Message'].unique()

array(['Program F_ThalaPseu_HalfNutr started',
       'Program F_ThalaPseu_HalfNutr ended',
       'program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65',
       'program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 50'],
      dtype=object)

In [19]:
only_feed.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
9,2021-4-20 13:4:7,1618916647,639887,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
15,2021-4-20 13:4:25,1618916665,177004,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
172,2021-4-20 14:5:54,1618920354,625413,2,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
1436,2021-4-20 22:0:5,1618948805,125813,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
1437,2021-4-20 22:0:6,1618948806,644921,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN


In [20]:
'''
(only_feed
 .assign(col=only_feed.groupby('Message').cumcount().add(1))
 .pivot(index='Message', columns='col', values='Time')
 .add_prefix('Time_')
 .reset_index().rename_axis(columns=None)
)
'''

"\n(only_feed\n .assign(col=only_feed.groupby('Message').cumcount().add(1))\n .pivot(index='Message', columns='col', values='Time')\n .add_prefix('Time_')\n .reset_index().rename_axis(columns=None)\n)\n"

In [21]:
#feed_reactor_instance4 = log_rawDF.query('Type == 2 and Parameter == "Reactor" and Time >= 1618561800 and Time <= 1625228995 and Unit =="FPC14"')

In [22]:
# manual_ins4 = log_rawDF.query('Time >= 1618561800 and Time <= 1625228995 and Unit =="FPC14" and Type == 3')

In [23]:
# only_feed
'''
for index, row in only_feed.iterrows():
    start_msg = row['Message']
    words = start_msg.split()
    check = "Vol"
    if check in words:
        new_msg = start_msg
    

for row in only_feed.itertuples():
    print(row.Type)
    print(row.Index)
'''



'\nfor index, row in only_feed.iterrows():\n    start_msg = row[\'Message\']\n    words = start_msg.split()\n    check = "Vol"\n    if check in words:\n        new_msg = start_msg\n    \n\nfor row in only_feed.itertuples():\n    print(row.Type)\n    print(row.Index)\n'

In [24]:
only_feed_newIndex = only_feed.reset_index()

In [25]:
only_feed_newIndex= only_feed_newIndex.drop('index',1)

In [26]:
only_feed_newIndex.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
0,2021-4-20 13:4:7,1618916647,639887,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
1,2021-4-20 13:4:25,1618916665,177004,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
2,2021-4-20 14:5:54,1618920354,625413,2,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
3,2021-4-20 22:0:5,1618948805,125813,2,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
4,2021-4-20 22:0:6,1618948806,644921,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN


In [27]:
'''
for idex, row in only_feed_newIndex.iterrows():
    start_msg = row['Message']
    words = start_msg.split()
    check = "Vol"
    if check in words:
        new_msg = start_msg

    check_str = "ended"
    if "ended" in start_msg:
        next_le = only_feed_newIndex.loc[idex+1]['Message']
        if "Vol" not in next_le:
            only_feed_newIndex.loc[idex+0.5] = new_msg
            
only_feed_newIndex = only_feed_newIndex.sort_index().reset_index(drop=True)
'''
    

'\nfor idex, row in only_feed_newIndex.iterrows():\n    start_msg = row[\'Message\']\n    words = start_msg.split()\n    check = "Vol"\n    if check in words:\n        new_msg = start_msg\n\n    check_str = "ended"\n    if "ended" in start_msg:\n        next_le = only_feed_newIndex.loc[idex+1][\'Message\']\n        if "Vol" not in next_le:\n            only_feed_newIndex.loc[idex+0.5] = new_msg\n            \nonly_feed_newIndex = only_feed_newIndex.sort_index().reset_index(drop=True)\n'

In [28]:
for idex, row in only_feed_newIndex.iterrows():
    start_msg = row['Message']
    words = start_msg.split()
    check = "Vol"
    if check in words:
        new_msg = start_msg

    check_str = "ended"
    if "ended" in start_msg:
        next_le = only_feed_newIndex.loc[idex+1]['Message']
        if "Vol" not in next_le:
            line = pd.DataFrame({"Message": new_msg}, index=[idex+0.5])
            only_feed_newIndex = only_feed_newIndex.append(line, ignore_index=False)
only_feed_newIndex = only_feed_newIndex.sort_index().reset_index(drop=True)

In [29]:
only_feed_newIndex.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
0,2021-4-20 13:4:7,1.618917e+09,639887.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
1,2021-4-20 13:4:25,1.618917e+09,177004.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
2,2021-4-20 14:5:54,1.618920e+09,625413.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
3,2021-4-20 22:0:5,1.618949e+09,125813.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
4,2021-4-20 22:0:6,1.618949e+09,644921.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN


In [30]:
for idex, row in only_feed_newIndex.iterrows():
    start_msg = row['Message']
    words = start_msg.split()
    check = "Vol"
    if check in words:
        new_msg = start_msg

    if (start_msg.endswith('started')):
        if idex == 0:
            continue
        else:
            pre_le = only_feed_newIndex.loc[idex-1]['Message']
            if "Vol" not in pre_le:
                line = pd.DataFrame({"Message": new_msg}, index=[(idex-1)+0.5])
                only_feed_newIndex = only_feed_newIndex.append(line, ignore_index=False)
                
only_feed_newIndex = only_feed_newIndex.sort_index().reset_index(drop=True)

In [31]:
only_feed_newIndex.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
0,2021-4-20 13:4:7,1.618917e+09,639887.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN
2,2021-4-20 13:4:25,1.618917e+09,177004.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN
3,2021-4-20 14:5:54,1.618920e+09,625413.0,2.0,FPC14,program,Program F_ThalaPseu_HalfNutr ended,NaN,NaN
4,2021-4-20 22:0:5,1.618949e+09,125813.0,2.0,FPC14,program,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65",NaN,NaN


In [32]:
only_feed_newIndex= only_feed_newIndex.drop(['MicroSec', 'Type', 'Value1', 'Value2', 'Parameter'],1)

In [33]:
only_feed_newIndex.head()

,Date_Time,Time,Unit,Message
0,2021-4-20 13:4:7,1.618917e+09,FPC14,Program F_ThalaPseu_HalfNutr started
1,NaN,NaN,NaN,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65"
2,2021-4-20 13:4:25,1.618917e+09,FPC14,Program F_ThalaPseu_HalfNutr started
3,2021-4-20 14:5:54,1.618920e+09,FPC14,Program F_ThalaPseu_HalfNutr ended
4,2021-4-20 22:0:5,1.618949e+09,FPC14,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65"


In [34]:
only_feed_newIndex['Date_Time'].fillna(method='bfill', inplace=True)
only_feed_newIndex['Time'].fillna(method='bfill', inplace=True)
only_feed_newIndex['Unit'].fillna(method='bfill', inplace=True)
only_feed_newIndex['Time'].fillna(method='bfill', inplace=True)


In [35]:
only_feed_newIndex.head()

,Date_Time,Time,Unit,Message
0,2021-4-20 13:4:7,1.618917e+09,FPC14,Program F_ThalaPseu_HalfNutr started
1,2021-4-20 13:4:25,1.618917e+09,FPC14,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65"
2,2021-4-20 13:4:25,1.618917e+09,FPC14,Program F_ThalaPseu_HalfNutr started
3,2021-4-20 14:5:54,1.618920e+09,FPC14,Program F_ThalaPseu_HalfNutr ended
4,2021-4-20 22:0:5,1.618949e+09,FPC14,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65"


In [36]:
clean_feedDF = only_feed_newIndex[only_feed_newIndex['Message'].str.contains('PrgID', regex=False, case=False, na=False)]

In [37]:
clean_feedDF.head()

,Date_Time,Time,Unit,Message
1,2021-4-20 13:4:25,1.618917e+09,FPC14,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65"
4,2021-4-20 22:0:5,1.618949e+09,FPC14,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65"
7,2021-4-21 11:23:12,1.618997e+09,FPC14,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65"
10,2021-4-21 22:0:5,1.619035e+09,FPC14,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65"
13,2021-4-22 10:0:5,1.619078e+09,FPC14,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65"


In [38]:
clean_feedDF.tail()

,Date_Time,Time,Unit,Message
382,2021-6-26 22:0:1,1.624738e+09,FPC14,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65"
384,2021-6-28 10:0:1,1.624867e+09,FPC14,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65"
387,2021-6-28 22:0:1,1.624910e+09,FPC14,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65"
389,2021-6-29 11:48:0,1.624960e+09,FPC14,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65"
392,2021-7-1 8:58:15,1.625123e+09,FPC14,"program auto started , PrgID = F_ThalaPseu_HalfNutr, Vol = 65"


In [39]:
clean_feedDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133 entries, 1 to 392
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date_Time  133 non-null    object 
 1   Time       133 non-null    float64
 2   Unit       133 non-null    object 
 3   Message    133 non-null    object 
dtypes: float64(1), object(3)
memory usage: 5.2+ KB


In [40]:
u = clean_feedDF['Message'].str.partition(" ")
pd.DataFrame({'Program': u[2].str.split().str[-4], 'Last': u[2].str.split().str[-1]})

,Program,Last
1,"F_ThalaPseu_HalfNutr,",65
4,"F_ThalaPseu_HalfNutr,",65
7,"F_ThalaPseu_HalfNutr,",65
10,"F_ThalaPseu_HalfNutr,",65
13,"F_ThalaPseu_HalfNutr,",65
...,...,...
382,"F_ThalaPseu_HalfNutr,",65
384,"F_ThalaPseu_HalfNutr,",65
387,"F_ThalaPseu_HalfNutr,",65
389,"F_ThalaPseu_HalfNutr,",65


In [41]:
clean_feedDF['program'] =  clean_feedDF['Message'].str.split().str[-4]
clean_feedDF['volume'] = clean_feedDF['Message'].str.split().str[-1]
clean_feedDF = clean_feedDF.drop('Message',1)

In [42]:
clean_feedDF['program'] = clean_feedDF['program'].str.replace(',', '')

In [43]:
clean_feedDF.tail()

,Date_Time,Time,Unit,program,volume
382,2021-6-26 22:0:1,1.624738e+09,FPC14,F_ThalaPseu_HalfNutr,65
384,2021-6-28 10:0:1,1.624867e+09,FPC14,F_ThalaPseu_HalfNutr,65
387,2021-6-28 22:0:1,1.624910e+09,FPC14,F_ThalaPseu_HalfNutr,65
389,2021-6-29 11:48:0,1.624960e+09,FPC14,F_ThalaPseu_HalfNutr,65
392,2021-7-1 8:58:15,1.625123e+09,FPC14,F_ThalaPseu_HalfNutr,65


In [44]:
clean_feedDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133 entries, 1 to 392
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date_Time  133 non-null    object 
 1   Time       133 non-null    float64
 2   Unit       133 non-null    object 
 3   program    133 non-null    object 
 4   volume     133 non-null    object 
dtypes: float64(1), object(4)
memory usage: 6.2+ KB


In [45]:
clean_feedDF = clean_feedDF.astype({'Date_Time':'datetime64[ns]', 'Time':'int64', 'volume':'float32'})

In [46]:
clean_feedDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133 entries, 1 to 392
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date_Time  133 non-null    datetime64[ns]
 1   Time       133 non-null    int64         
 2   Unit       133 non-null    object        
 3   program    133 non-null    object        
 4   volume     133 non-null    float32       
dtypes: datetime64[ns](1), float32(1), int64(1), object(2)
memory usage: 5.7+ KB


In [47]:
clean_feedDF.head()

,Date_Time,Time,Unit,program,volume
1,2021-04-20 13:04:25,1618916665,FPC14,F_ThalaPseu_HalfNutr,65.0
4,2021-04-20 22:00:05,1618948805,FPC14,F_ThalaPseu_HalfNutr,65.0
7,2021-04-21 11:23:12,1618996992,FPC14,F_ThalaPseu_HalfNutr,65.0
10,2021-04-21 22:00:05,1619035205,FPC14,F_ThalaPseu_HalfNutr,65.0
13,2021-04-22 10:00:05,1619078405,FPC14,F_ThalaPseu_HalfNutr,65.0


In [48]:
product_temp_ins4 = log_rawDF.query('Time >= 1618561800 and Time <= 1625228995 and Unit =="Product" and Parameter == "Recipes"')

In [49]:
product_temp_ins4.tail(60)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
42981,2021-5-4 14:21:38,1620130898,328536,3,Product,Recipes,Recipe Saved - Skeletonema marinoi,NaN,NaN
44605,2021-5-5 14:15:2,1620216902,408366,3,Product,Recipes,Recipe Saved - Diacronema lutheri (Pavlova),NaN,NaN
84741,2021-6-4 12:42:22,1622803342,17012,3,Product,Recipes,Recipe Saved - NaOH_0.1M_rinse,NaN,1 CompWater 500.02 NaOH_1M 100.03 EDTA_100mM 0.04 CompWater 400.0
84743,2021-6-4 12:42:28,1622803348,786999,3,Product,Recipes,Recipe Step deleted - NaOH_0.1M_rinse,1 CompWater 500.02 NaOH_1M 100.03 EDTA_100mM 0.04 CompWater 400.0,1 CompWater 500.02 NaOH_1M 100.04 CompWater 400.0


In [57]:
programs = clean_feedDF['program'].unique()
programs

array(['F_ThalaPseu_HalfNutr'], dtype=object)

In [58]:
prgm = list(programs)

In [60]:
def dbConn_sqlAlch():
    try:
        database_username = 'root'
        database_password = 'password'
        database_ip       = '127.0.0.1:3306'
        database_name     = 'data_dashboard'
        database_connection = sqlalchemy.create_engine('mariadb+mariadbconnector://{0}:{1}@{2}/{3}'.
                                                    format(database_username, database_password, 
                                                            database_ip, database_name))
        return database_connection
    except Exception as e: print(e)


database_connection = dbConn_sqlAlch()
    

In [64]:
# check DB connection is fine and fetch unique uint ID
feed_program = pd.read_sql('SELECT * FROM tbl_program_product WHERE ProgramID = ?', database_connection, params=[prgm[0]])
feed_program.head(10)

,ProgramID,Product,FeedVolume
0,F_ThalaPseu_HalfNutr,Complete 3_5de half,60.0
1,F_ThalaPseu_HalfNutr,FeedWaterAcid,30.0
2,F_ThalaPseu_HalfNutr,Si_1_5de,10.0


In [73]:
products = feed_program['Product'].unique()
prd = list(products)
prd

['Complete 3_5de half', 'FeedWaterAcid', 'Si_1_5de']

In [86]:
df = pd.read_sql('SELECT * FROM tbl_recipes_product WHERE prodname = ?', database_connection, params=[prd[0]])

In [87]:
df.head(10)

,prodname,compname,conc
0,Complete 3_5de half,CompWater,687.60
1,Complete 3_5de half,NaNO3_2.47M,16.70
2,Complete 3_5de half,"NaCl_4,3M",158.33
3,Complete 3_5de half,KCl_1M,16.70
4,Complete 3_5de half,MgSO4_2M,16.70
5,Complete 3_5de half,CaCl2 0.67M,10.50
6,Complete 3_5de half,"H3B03_0,0625M",10.00
7,Complete 3_5de half,Micronutrients 400xF,16.70
8,Complete 3_5de half,FeSO4/EDTA 12.05 mM,16.70
9,Complete 3_5de half,Vitamins 2000xF,13.37


In [89]:
df_pivot = df.pivot_table(index=(['prodname']),columns='compname',values='conc',fill_value=0)
df_pivot = df_pivot.reset_index()

In [90]:
df_pivot.head()

compname,prodname,CaCl2 0.67M,CompWater,FeSO4/EDTA 12.05 mM,"H3B03_0,0625M",KCl_1M,KH2PO4_0.157M,MgSO4_2M,Micronutrients 400xF,Na2SeO3 0.001 mM,"NaCl_4,3M",NaNO3_2.47M,Vitamins 2000xF
0,Complete 3_5de half,10.5,687.6,16.7,10,16.7,16.7,16.7,16.7,20,158.33,16.7,13.37


In [91]:
recipe = "recipe"
recipe_list = []
for product_item in prd:
    df = pd.read_sql('SELECT * FROM tbl_recipes_product WHERE prodname = ?', database_connection, params=[product_item])
    df_pivot = df.pivot_table(index=(['prodname']),columns='compname',values='conc',fill_value=0)
    df_pivot = df_pivot.reset_index()
    recipe_list.append(df_pivot)

In [97]:
recipe_list[2]['prodname']

0    Si_1_5de
Name: prodname, dtype: object